In [54]:
from collections import UserDict
from re import U
import pandas as pd

matches = pd.read_csv("matches.csv", index_col=0)
matches.head()

matches[matches["team"] == "Liverpool"]

#liverpool is missing data for 2021-2022 season

matches["round"].value_counts()

#cleaning data phase


matches["date"] = pd.to_datetime(matches["date"])

matches.dtypes

#creating predictors, change venue column to 0,1

matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek

#target: whether the team won or not
matches["target"] = (matches["result"] == "W").astype("int")
matches

#ML Model w/ random forestclassifier

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
#split the data into training training data should be ME dated earlier than text, testing,
train = matches[matches["date"] <'2022-01-01']
test = matches[matches["date"] > '2022-01-01']
predictors = ["venue_code", "opp_code", "hour", "day_code"]

#fit random forest model unto predictor columns, and training DB
rf.fit(train[predictors],train["target"])

preds = rf.predict(test[predictors])
from sklearn.metrics import accuracy_score
acc = accuracy_score(test["target"], preds)
acc
#display accuracy metrics
combined = pd.DataFrame(dict(actual=test["target"],predictions=preds))
pd.crosstab(index=combined["actual"], columns=combined["predictions"])

#after viewing Wpred percentage is sig. low, to improve precision with rolling avgs
from sklearn.metrics import precision_score
precision_score(test["target"],preds)
#groupmatches is a set of DataFrames by group (groupedby team)
grouped_matches = matches.groupby("team")
group = grouped_matches.get_group("Manchester City")
group

def rolling_averages(group,cols, new_cols):
  group = group.sort_values("date")
  rolling_stats = group[cols].rolling(3,closed='left').mean()
  group[new_cols] = rolling_stats
  group = group.dropna(subset=new_cols)
  return group
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]
rolling_averages(group, cols, new_cols)
#create new dataframe with rolling averages applied to each and an index level is dropped for better level access
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling
matches_rolling.index = range(matches_rolling.shape[0])

#retraining the ML model
def make_predictions(data, predictors):
  train = data[data["date"] <'2022-01-01']
  test = data[data["date"] > '2022-01-01']
  rf.fit(train[predictors],train["target"])
  preds = rf.predict(test[predictors])
  combined = pd.DataFrame(dict(actual=test["target"],predictions=preds))
  precision = precision_score(test["target"], preds)
  return combined, precision

  #!!!!31:0#call new formatted prediction function with new cols added to set of predictors
combined, precision = make_predictions(matches_rolling, predictors + new_cols)
combined = combined.merge(matches_rolling[["date","team","opponent","result"]], left_index=True, right_index=True)

#combining home and away predictions
#fixing uncharacteristic team name entry with MissingDict and dictionary with unique changes
class MissingDict(dict):
  __missing__ = lambda self, key: key

map_values = {
    "Brighton and Move Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcastle Utd",
    "Tottenham Hotspur": "Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves"
}

mapping = MissingDict(**map_values)
mapping["Arsenal"]

combined["new_team"] = combined["team"].map(mapping)
#look for new team field, merge it with opponent field on opp side
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])
merged


,actual_x,predictions_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predictions_y,team_y,opponent_y,result_y,new_team_y
0,0,0,2022-01-23,Arsenal,Burnley,D,Arsenal,0,0,Burnley,Arsenal,D,Burnley
1,1,0,2022-02-10,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
2,1,0,2022-02-19,Arsenal,Brentford,W,Arsenal,0,0,Brentford,Arsenal,L,Brentford
3,1,1,2022-02-24,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
4,1,1,2022-03-06,Arsenal,Watford,W,Arsenal,0,0,Watford,Arsenal,L,Watford
...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W,Wolves,0,0,Everton,Wolves,L,Everton
243,0,0,2022-03-18,Wolverhampton Wanderers,Leeds United,L,Wolves,1,0,Leeds United,Wolves,W,Leeds United
244,1,0,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,Wolves,0,0,Aston Villa,Wolves,L,Aston Villa
245,0,0,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,Wolves,1,0,Newcastle United,Wolves,W,Newcastle Utd
